In [1]:
%reload_ext autoreload

%autoreload 2

In [2]:
from Libraries_functions import *
from tqdm import tqdm
from IPython.display import Audio 
import ipywidgets as widgets
from IPython.display import display
from pesq import pesq
import glob

In [4]:
model = keras.models.load_model('./models/BestBUENO3996_noise_lr_0.001_btch_16_epch_50_loss_mae_opt_Adam.h5')

In [ ]:
style = {'description_width': 'initial'}

select= widgets.Dropdown(
    options=[('noise', 0), ('voice', 1)],
    value=0,
    description='Select noise or voice for comparing:',
    style=style,
    layout={'width': 'max-content'}
)
make_pred= widgets.Dropdown(
    options=[('save', 0), ('not save', 1)],
    value=1,
    description='Select save or not save pred audios:',
    style=style,
    layout={'width': 'max-content'}
)

## Cargamos el valor de lo que vamos a predecir

In [5]:
ruta="../../Datasets/Test/mixed_sound"
class_r=os.listdir(ruta)
mix_real=[]
scaler_total=[]
for i,entry in enumerate(class_r):
        type_noise = ruta+'/'+entry
        mix_real.append(type_noise)

X_test=[]

for i in mix_real:
        signal_noise, sr_noise = librosa.load(i, sr=SAMPLE_RATE)
        mix_wav, mix_wav_phase = librosa.magphase(librosa.stft(signal_noise, n_fft=N_FFT, hop_length=HOP_LENGTH))
        mel_spec= librosa.feature.melspectrogram(S=mix_wav,sr=SAMPLE_RATE,n_fft=N_FFT,hop_length=HOP_LENGTH)
        X_test.append(mel_spec)

## Creamos varias variables porque no es posible si no, luego hacer la inversion de la normalización
# scalerX1 = MinMaxScaler()
# scalerX2 = MinMaxScaler()
# scalerX3 = MinMaxScaler()
# scalerX4 = MinMaxScaler()
for i in range(len(X_test)):
        pepe=MinMaxScaler()
        scaler_total.append(pepe)

for i in range(len(X_test)):
        X_test[i]=scaler_total[i].fit_transform(X_test[i])
# X_test[0]=scalerX1.fit_transform(X_test[0])
# X_test[1]=scalerX2.fit_transform(X_test[1])
# X_test[2]=scalerX3.fit_transform(X_test[2])
# X_test[3]=scalerX4.fit_transform(X_test[3])

# scaler_total=[scalerX1,scalerX2,scalerX3,scalerX4]

In [6]:
np.shape(X_test)

(200, 128, 192)

In [7]:
X_test = np.reshape(X_test, (np.shape(X_test)[0], np.shape(X_test)[1],np.shape(X_test)[2],1))


In [8]:
np.shape(X_test)

(200, 128, 192, 1)

In [9]:

pred_outp=model.predict(X_test)
pred_outp=np.squeeze(pred_outp)


In [10]:
np.shape(pred_outp)


(200, 128, 192)

In [12]:
display(select)
display(make_pred)

Dropdown(description='Select noise or voice for comparing:', layout=Layout(width='max-content'), options=(('no…

Dropdown(description='Select save or not save pred audios:', index=1, layout=Layout(width='max-content'), opti…

In [13]:
ruta_noise="../../Datasets/Test/noise/"
class_noise=os.listdir(ruta_noise)
ruta_voice="../../Datasets/Test/voice/"
class_voice=os.listdir(ruta_voice)
real=[]
if select.value==0:
    for i,entry in enumerate(class_noise):
        type_noise = ruta_noise+'/'+entry
        real.append(type_noise)
else:
    for i,entry in enumerate(class_voice):
        type_noise = ruta_voice+'/'+entry
        real.append(type_noise)
y_test=[]

for i in real:
    signal_noise, sr_noise = librosa.load(i, sr=SAMPLE_RATE)
    mix_wav, mix_wav_phase = librosa.magphase(librosa.stft(signal_noise, n_fft=N_FFT, hop_length=HOP_LENGTH))
    mel_spec= librosa.feature.melspectrogram(S=mix_wav,sr=SAMPLE_RATE,n_fft=N_FFT,hop_length=HOP_LENGTH)
    y_test.append(mel_spec)

for i in range(len(X_test)):
        scalerY = MinMaxScaler()
        y_test[i]=scalerY.fit_transform(y_test[i])

# Métricas de los audios

In [16]:
##cargamos los arrays de datos
AudioName = "./results/Convertido-mixed/convertido_mel"
array_pred=[]
array_original=[]
for i in tqdm(range(len(pred_outp))):

    noise_real_signal, sr_mixed = librosa.load(mix_real[i], sr=SAMPLE_RATE)

    out_wav=scaler_total[i].inverse_transform(pred_outp[i])

    mix_wav, mix_wav_phase = librosa.magphase(librosa.stft(noise_real_signal, n_fft=N_FFT, hop_length=HOP_LENGTH))

    out_wav=librosa.feature.inverse.mel_to_stft(out_wav,sr=SAMPLE_RATE,n_fft=N_FFT)

    out_wav,magphase=librosa.magphase(out_wav)

    array_pred.append(librosa.core.istft(
    out_wav * mix_wav_phase
    , win_length=2048, hop_length=HOP_LENGTH))
    
    if(i==73):
        print(mix_real[73])
    
        sf.write(f'{AudioName}_{i}.wav',array_pred[73], SAMPLE_RATE)


for i in real:
    signal_noise, sr_noise = librosa.load(i, sr=SAMPLE_RATE)
    array_original.append(signal_noise)

 37%|███▋      | 74/200 [03:56<11:09,  5.31s/it]

../../Datasets/Test/mixed_sound/mixed_sound2344.wav


100%|██████████| 200/200 [15:29<00:00,  4.65s/it]


### MSE

In [17]:
aircrafts=[]
lluvia=[]
trafico=[]
viento=[]

array_original=np.array(array_original)
array_original=array_original[:,:95882]

for i in tqdm(range(len(pred_outp))):
    if(i<len(pred_outp)/4):
        aircrafts.append(mean_squared_error(array_pred[i],array_original[i] ,squared=True))
    elif(i>=len(pred_outp)/4 and i<(len(pred_outp)/4)*2):
        lluvia.append(mean_squared_error(array_pred[i], array_original[i],squared=True))
    elif(i>=(len(pred_outp)/4)*2 and i<(len(pred_outp)/4)*3):
        trafico.append(mean_squared_error(array_pred[i],array_original[i],squared=True))
    else:
        viento.append(mean_squared_error(array_pred[i], array_original[i],squared=True))

100%|██████████| 200/200 [00:00<00:00, 2560.00it/s]


In [18]:
df=pd.DataFrame()
df.insert(0,"aircrafts_mean",[mean(aircrafts)])
df.insert(1,"aircrafts_desviacion",[desviacion(aircrafts)])

df.insert(2,"lluvia_mean",[mean(lluvia)])
df.insert(3,"lluvia_desviacion",[desviacion(lluvia)])

df.insert(4,"trafico_mean",[mean(trafico)])
df.insert(5,"trafico_desviacion",[desviacion(trafico)])

df.insert(6,"viento_mean",[mean(viento)])
df.insert(7,"viento_desviacion",[desviacion(viento)])

In [19]:
df

,aircrafts_mean,aircrafts_desviacion,lluvia_mean,lluvia_desviacion,trafico_mean,trafico_desviacion,viento_mean,viento_desviacion
0,0.007017,0.007674,0.005524,0.006064,0.006659,0.012327,0.016145,0.023509


### PESQ

In [22]:
from pesq import pesq
exception=0
aircrafts=[]
lluvia=[]
trafico=[]
viento=[]

for i in tqdm(range(len(pred_outp))):
    try:
        if(i<len(pred_outp)/4):
            aircrafts.append(pesq(16000, array_pred[i], array_original[i],'wb'))
        elif(i>=len(pred_outp)/4 and i<(len(pred_outp)/4)*2):
            lluvia.append(pesq(16000, array_pred[i], array_original[i],'wb'))
        elif(i>=(len(pred_outp)/4)*2 and i<(len(pred_outp)/4)*3):
            trafico.append(pesq(16000, array_pred[i], array_original[i],'wb'))
        else:
            viento.append(pesq(16000, array_pred[i], array_original[i],'wb'))
    except:
        exception+=1

print(exception)

100%|██████████| 200/200 [00:31<00:00,  6.40it/s]


In [24]:
df_PESQ=pd.DataFrame()
df_PESQ.insert(0,"aircrafts_mean",[mean(aircrafts)])
df_PESQ.insert(1,"aircrafts_desviacion",[desviacion(aircrafts)])

df_PESQ.insert(2,"lluvia_mean",[mean(lluvia)])
df_PESQ.insert(3,"lluvia_desviacion",[desviacion(lluvia)])

df_PESQ.insert(4,"trafico_mean",[mean(trafico)])
df_PESQ.insert(5,"trafico_desviacion",[desviacion(trafico)])

df_PESQ.insert(6,"viento_mean",[mean(viento)])
df_PESQ.insert(7,"viento_desviacion",[desviacion(viento)])

In [25]:
df_PESQ

,aircrafts_mean,aircrafts_desviacion,lluvia_mean,lluvia_desviacion,trafico_mean,trafico_desviacion,viento_mean,viento_desviacion
0,1.351499,0.425087,1.318228,0.558174,1.349133,0.428863,1.630061,0.77877


### SNR

In [26]:
exception=0
aircrafts=[]
lluvia=[]
trafico=[]
viento=[]

for i in tqdm(range(len(pred_outp))):
    try:
        if(i<len(pred_outp)/4):
            aircrafts.append(SNRsystem(array_original[i],array_pred[i]))
        elif(i>=len(pred_outp)/4 and i<(len(pred_outp)/4)*2):
             lluvia.append(SNRsystem(array_original[i],array_pred[i]))
        elif(i>=(len(pred_outp)/4)*2 and i<(len(pred_outp)/4)*3):
            trafico.append(SNRsystem(array_original[i],array_pred[i]))
        else:
            viento.append(SNRsystem(array_original[i],array_pred[i]))
    except:
        exception+=1

print(exception)

100%|██████████| 200/200 [00:03<00:00, 50.00it/s]

150


In [27]:
df_SNR=pd.DataFrame()
df_SNR.insert(0,"aircrafts_mean",[mean(aircrafts)])
df_SNR.insert(1,"aircrafts_desviacion",[desviacion(aircrafts)])

df_SNR.insert(2,"lluvia_mean",[mean(lluvia)])
df_SNR.insert(3,"lluvia_desviacion",[desviacion(lluvia)])

df_SNR.insert(4,"trafico_mean",[mean(trafico)])
df_SNR.insert(5,"trafico_desviacion",[desviacion(trafico)])

df_SNR.insert(6,"viento_mean",[mean(viento)])
df_SNR.insert(7,"viento_desviacion",[desviacion(viento)])

In [28]:
df_SNR

,aircrafts_mean,aircrafts_desviacion,lluvia_mean,lluvia_desviacion,trafico_mean,trafico_desviacion,viento_mean,viento_desviacion
0,-4.07703,2.494734,-7.066224,4.124708,-6.907141,5.215686,-7.540713,4.29635
